In [37]:
from datetime import datetime

In [38]:
TODAY = datetime.today().strftime('%Y-%m-%d')
HDFS_RUCIO_DIDS = f"/project/awg/cms/rucio/{TODAY}/dids/part*.avro"
HDFS_RUCIO_CONTENTS = f"/project/awg/cms/rucio/{TODAY}/contents/part*.avro"
HDFS_RUCIO_RULES = f"/project/awg/cms/rucio/{TODAY}/rules/part*.avro"
HDFS_RUCIO_LOCKS = f"/project/awg/cms/rucio/{TODAY}/contents/part*.avro"

def get_df_dids(spark):
    return spark.read.format("avro").load(HDFS_RUCIO_DIDS) \
        .withColumnRenamed("BYTES", "SIZE") \
        .select(["name", "scope", "account", "did_type", "is_open", "monotonic", "availability", "size", "length"])




def get_df_contents(spark):
     return spark.read.format("avro").load(HDFS_RUCIO_CONTENTS) \
        .select(["name", "did_type", "child_name", "child_type"])

In [39]:
def get_df_dids(spark):
    return spark.read.format("avro").load(HDFS_RUCIO_DIDS) \
        .withColumnRenamed("BYTES", "SIZE") \
        .select(["name", "scope", "account", "did_type", "is_open", "monotonic", "availability", "size", "length"])




def get_df_contents(spark):
     return spark.read.format("avro").load(HDFS_RUCIO_CONTENTS) \
        .select(["name", "did_type", "child_name", "child_type"])

## File, Dataset and Container Nodes

In [40]:
df_contents = get_df_contents(spark)

In [41]:
df_did = get_df_dids(spark)
df_container  = df_did.filter(df_did.did_type == 'C')
df_dataset = df_did.filter(df_did.did_type == 'D')
df_file = df_did.filter(df_did.did_type == 'F')



In [42]:
#Using 1000 containers
df_container_table = df_container.limit(1000)
container_names = [row.name for row in df_container_table.select('name').collect()]

In [43]:
# print(f"{dataset_names.count()=}")
print(f"{df_container_table.count()=}")
print(f"{df_contents.count()=}")

df_container_table.count()=1000
df_contents.count()=101024788


In [44]:
#Collecting data sets that are part of these 1000 containers
dataset_names = df_container_table.join(df_contents, df_container_table["name"]==df_contents["name"], "inner").select('child_name')
print(f"{dataset_names.count()=}")

dataset_names.count()=7656


In [13]:
print(f"{dataset_names.count()=}")


dataset_names.count()=34624


In [14]:
print(f"{dataset_names.count()=}")

dataset_names.count()=1500


In [45]:
df_dataset_table = df_dataset.join(dataset_names, df_dataset.name==dataset_names.child_name, "inner")


In [16]:
print(f"{dataset_names.count()=}")


dataset_names.count()=3924


In [28]:
df_dataset_table.limit(2).toPandas()

,name,scope,account,did_type,is_open,monotonic,availability,size,length,child_name
0,/BTagMu/Run2018D-12Nov2019_UL2018-v1/AOD#57869...,cms,sync_t1_ru_jinr_disk,D,0,0,A,477684518971,119,/BTagMu/Run2018D-12Nov2019_UL2018-v1/AOD#57869...
1,/BTagMu/Run2018D-12Nov2019_UL2018-v1/AOD#e7a64...,cms,sync_t1_ru_jinr_disk,D,0,0,A,1960372358,1,/BTagMu/Run2018D-12Nov2019_UL2018-v1/AOD#e7a64...


In [46]:
from pyspark.sql.functions import col

#Collecting files that are part of above datasets
file_names = df_dataset_table.alias("temp_df_name").select("name").join(df_contents.alias("df_contents2"), col("temp_df_name.name")==col("df_contents2.name"), "inner").select('child_name')
# file_names = [row.child_name for row in file_names]
df_file_table = df_file.join(file_names, df_file.name==file_names.child_name, "inner")

In [47]:
df_file_table.count()

30

In [16]:

#Write container nodes to neo4j
results = df_container_table.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("batch.size", 500)\
  .option("url", "bolt://r-neo4j.cern.ch:7687")\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "mypass")\
  .option("labels", ":Container")\
  .option("node.keys", "name")\
  .option("schema.optimization.type", "INDEX")\
  .save()


In [17]:
#Write dataset nodes to neo4j
results = df_dataset_table.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("batch.size", 500)\
  .option("url", "bolt://r-neo4j.cern.ch:7687")\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "mypass")\
  .option("labels", ":Dataset")\
  .option("node.keys", "name")\
  .option("schema.optimization.type", "INDEX")\
  .save()

22/06/20 10:51:29 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [12]:
#Write file nodes to neo4j
results = df_file_table.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("batch.size", 500)\
  .option("url", "bolt://r-neo4j.cern.ch:7687")\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "mypass")\
  .option("labels", ":File")\
  .option("node.keys", "name")\
  .option("schema.optimization.type", "INDEX")\
  .save()

22/06/20 11:03:45 ERROR Utils: uncaught error in thread spark-listener-group-eventLog, stopping SparkContext
java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOfRange(Arrays.java:3664)
	at java.lang.String.<init>(String.java:207)
	at java.lang.StringBuilder.toString(StringBuilder.java:407)
	at com.fasterxml.jackson.core.util.TextBuffer.contentsAsString(TextBuffer.java:426)
	at com.fasterxml.jackson.core.io.SegmentedStringWriter.getAndClear(SegmentedStringWriter.java:83)
	at com.fasterxml.jackson.databind.ObjectMapper.writeValueAsString(ObjectMapper.java:3410)
	at org.json4s.jackson.JsonMethods.compact(JsonMethods.scala:39)
	at org.json4s.jackson.JsonMethods.compact$(JsonMethods.scala:39)
	at org.json4s.jackson.JsonMethods$.compact(JsonMethods.scala:55)
	at org.apache.spark.scheduler.EventLoggingListener.logEvent(EventLoggingListener.scala:100)
	at org.apache.spark.scheduler.EventLoggingListener.onOtherEvent(EventLoggingListener.scala:274)
	at org.apache.spark.schedul

Py4JJavaError: An error occurred while calling o125011.save.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)

T

## Consists of relationship


In [ ]:
df_container_dataset_relation_table = df_contents\
    .filter(df_contents.name.isin(container_names))\
    .filter(df_contents.did_type=='C')\
    .filter(df_contents.child_type=='D')\
    .select(['name', 'child_name'])\
    .withColumnRenamed("name", "source.name")\
    .withColumnRenamed("child_name", "target.name")

df_dataset_file_relation_table = df_contents\
    .filter(df_contents.name.isin(dataset_names))\
    .filter(df_contents.did_type=='D')\
    .filter(df_contents.child_type=='F')\
    .select(['name', 'child_name'])\
    .withColumnRenamed("name", "source.name")\
    .withColumnRenamed("child_name", "target.name")